In [1]:
INPUTFILE = 'input-7-test.txt'

with open(INPUTFILE, 'r') as file:
    input = file.read()
rows = input.split("\n")

In [2]:
stupid_size = 0
for row in rows:
  command = row.split(" ")
  if command[0].isnumeric():
    stupid_size += int(command[0])

print(stupid_size)

48381621


In [3]:
from functools import reduce  # forward compatibility for Python 3
import operator
from pprint import pprint

def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

In [4]:
system = {"/" : {}}
current_dir = []
#history_of_cwd = []
for row in rows:
  command = row.split(" ")
  if command[0] == "$":
    if command[1] == "cd":
      if command[2] == "..":
        current_dir.pop()
      else:
        current_dir.append(command[2])
    elif command[1] == "ls":
      pass
    else:
      print(f"problem: unprocessed command {command}")
  else:
    if command[0] == "dir":
      #if any(map(str.isdigit, command[1])):
      #  print(f"problem: digit in dir name {command}")
      setInDict(dataDict=system, mapList=current_dir + [command[1]], value= {})
    elif command[0].isnumeric():
      setInDict(dataDict=system, mapList=current_dir + [command[1]], value=command[0])
    else:
      print(f"problem: unprocessed line {command}")


In [6]:
system

{'/': {'a': {'e': {'i': '584', 'asdf': '456'},
   'f': '29116',
   'g': '2557',
   'h.lst': '62596'},
  'b.txt': '14848514',
  'c.dat': '8504156',
  'd': {'j': '4060174',
   'd.log': '8033020',
   'd.ext': '5626152',
   'k': '7214296'}}}

In [7]:
def dict_size(dataDict):
  total_size = 0
  for key in dataDict:
    if type(dataDict[key]) == dict:
      total_size += dict_size(dataDict=dataDict[key])
    else:
      total_size += int(dataDict[key])
  return total_size

In [8]:
dict_size(system)

48381621

In [10]:
def add_dict_size(dataDict, dir_sizes: dict, path: str = ""):
  for key in dataDict:
    if type(dataDict[key]) == dict:
      path += key
      if key in dir_sizes:
        print(f"problem: dirname {path} already exists")
      dir_sizes[path] = dict_size(dataDict[key])
      add_dict_size(dataDict=dataDict[key], dir_sizes=dir_sizes, path=path)
  return dir_sizes

In [11]:
dir_sizes = {}
dir_sizes = add_dict_size(dataDict=system, dir_sizes=dir_sizes)

total_size_sum_max_1e5 = 0
for key in dir_sizes:
  if dir_sizes[key] <= 1e5:
    total_size_sum_max_1e5 += dir_sizes[key]
print(total_size_sum_max_1e5)

96349


In [59]:
dict_size(system)

42143088

Find smallest dictionary that frees up enough space for the update

In [12]:
total_disk_space = 70000000
update_space = 30000000
free_space = total_disk_space - dict_size(system)
needed_space = update_space - free_space

In [13]:
best_candidate = total_disk_space
for key in dir_sizes:
  if dir_sizes[key] > needed_space and dir_sizes[key] < best_candidate:
    best_candidate = dir_sizes[key]
print(best_candidate)

24933642
